<a href="https://colab.research.google.com/github/shreyas-muralidhara/Enron-Email-Priority-Sorting-Response-Recommendation/blob/main/Email_Page_rank.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Page Ranking for Enron Email Dataset

CSC791 - Natural Language Processing Spring 2020

Author - Shreyas Chikkballapur Muralidhara - schikkb  
          Sharath Narayana - snaraya9
         

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Import Libraries

In [ ]:
!tar -xvf '/content/drive/Shared drives/NLP Project/enron_mail_20150507.tar.gz' -C '/content/drive/Shared drives/NLP Project/'

In [ ]:
import pandas as pd
import numpy as np
import pandas as pd
import xlrd
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
import re
import gensim

from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, f1_score,accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer,TfidfTransformer

from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC,LinearSVC

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


Load the dataset into a dataframe

In [ ]:
email_df = pd.read_csv('/content/drive/Shared drives/NLP Project/test.csv',index_col=None,header=0)
email_df.Date_list = email_df.Date_list.str.replace(',','')
print(email_df.shape)

(9571, 7)
["kmann (Non-Privileged).pst\n\n15:37:55 Synchronizing Mailbox 'Mann, Kay'\n15:37:55 Synchronizing Hierarchy\n15:37:56 Synchronizing Favorites\n15:37:56 Synchronizing Folder 'Inbox'\n15:38:12 "]
ynchronization Log:


Extracting Profile Information

In [ ]:
# Consider the necessary columns, without the category & labels
features_df = email_df



# Select the top  receivers of emails, extract the emails sent by them - 
# 1.SYMES-K(kate.symes@enron.com)
# 2.DASOVICH-J(jeff.dasovich@enron.com)
# 3.MANN-K(kay.mann@enron.com)

#Extracting Profile 1
#extract the emails that were sent to SYMES-K and received by SYMES-K
# SYMESK_df = features_df[features_df['To'].str.contains('kate.symes@enron.com', na=False)]
# SYMESK_df = SYMESK_df.append(features_df[features_df['From'].str.contains('kate.symes@enron.com', na=False)], ignore_index=True)
# print(SYMESK_df.shape)

# #Extracting Profile 2
# #extract the emails that were sent to DASOVICH-J and received by DASOVICH-J
# DASOVICHJ_df = features_df[features_df['To'].str.contains('jeff.dasovich@enron.com', na=False)]
# DASOVICHJ_df = DASOVICHJ_df.append(features_df[features_df['From'].str.contains('jeff.dasovich@enron.com', na=False)], ignore_index=True)
# print(DASOVICHJ_df.shape)

#Extracting Profile 3
#extract the emails that were sent to MANN-K and received by MANN-K
MANNK_df = features_df[features_df['To_list'].str.contains('kay.mann@enron.com', na=False)]
MANNK_df = MANNK_df.append(features_df[features_df['From_list'].str.contains('kay.mann@enron.com', na=False)], ignore_index=True)
print(MANNK_df.shape)

#features_df.groupby(['Subject'])['Subject'].count().reset_index(name='count').sort_values(['count'], ascending=False).head(50)# Cat_1_level_2.unique()


(8647, 7)


Page Ranking 


In [ ]:
import math
import numpy as np
class PageRank:

	def __init__(self, beta, edges, epsilon, max_iterations, node_num, weight_vector):
		self.beta = beta
		self.edges = edges
		self.epsilon = epsilon
		self.node_num = node_num
		self.MAX_ITERATIONS = max_iterations
		self.weight_vector = weight_vector

	def pageRank(self):
		final_rank_vector = np.zeros(self.node_num)
		initial_rank_vector = np.fromiter(
			[1 / self.node_num for _ in range(self.node_num)], dtype='float')
		iterations = 0
		diff = math.inf
		while(iterations < self.MAX_ITERATIONS and diff > self.epsilon):
			new_rank_vector = np.zeros(self.node_num)
			for parent in self.edges:
				for child in self.edges[parent]:
					new_rank_vector[child] += (self.weight_vector[(parent,child)] / len(self.edges[parent]))

			leaked_rank = (1-sum(new_rank_vector))/self.node_num
			final_rank_vector = new_rank_vector + leaked_rank
			diff = sum(abs(final_rank_vector - initial_rank_vector))
			initial_rank_vector = final_rank_vector
			iterations += 1
		return final_rank_vector

In [ ]:
import numpy as np
from collections import defaultdict

features_df['From'] = features_df['From_list']
features_df['To'] = features_df['To_list']

temp_df = features_df.groupby(['From','To'])['To'].count().reset_index(name='count').sort_values(['count'], ascending=False).head(50)# Cat_1_level_2.unique()
# temp_df = temp_df.map(lambda line: line.From != line.To)
comparison_column = np.where(temp_df['From'] != temp_df['To'], True, False)
temp_df['equal'] = comparison_column
temp_df = temp_df.loc[temp_df['equal'] == True].head(20)
from_users = temp_df['From'].to_list()
to_users = temp_df['To'].to_list()
node_num_dict = {}

temp_list = to_users+from_users
count = 1
user_list = []

#Create User Dictionary 
for user in temp_list:
  if user not in node_num_dict:
    node_num_dict[user] = count 
    user_list.append(user)
    count += 1

weight_dict = {}
edge_list = []
edges = defaultdict(list)
weight_list = []

#Create weighted dictionary 
for i in range(len(user_list)):
  for j in range(i+1,len(user_list)):
    if i == j:
      continue
    else:
      edge_df = features_df[(features_df['From']==user_list[i]) & (features_df['To']==user_list[j])]
      temp_count = edge_df['From'].count()
      if temp_count != 0 :
        weight_dict[(node_num_dict[user_list[i]], node_num_dict[user_list[j]])] = temp_count
        weight_list.append(temp_count)
        edge_list.append((node_num_dict[user_list[i]], node_num_dict[user_list[j]]))
        edges[node_num_dict[user_list[i]]].append(node_num_dict[user_list[j]])

      reverse_edge_df = features_df[(features_df['From']==user_list[j]) & (features_df['To']==user_list[i])]
      temp_reverse_count = reverse_edge_df['From'].count()
      if temp_reverse_count != 0 :
        weight_dict[(node_num_dict[user_list[j]], node_num_dict[user_list[i]])] = temp_reverse_count
        weight_list.append(temp_reverse_count)
        edge_list.append((node_num_dict[user_list[j]], node_num_dict[user_list[i]]))
        edges[node_num_dict[user_list[j]]].append(node_num_dict[user_list[i]])

#Normalize weighted dictionary
def NormalizeDict(wd,weight_list):
    max_value = max(weight_list)
    min_value = min(weight_list)
    for node in weight_dict:
      wd[node] = (wd[node]-min_value)/(max_value-min_value)
    return wd

weight_dict = NormalizeDict(weight_dict,weight_list)

Clean data and weight ranking framework


In [ ]:
import re

def clean_data(df):
  preprocessed_Subject, preprocessed_content = [], []

  for index, row in df.iterrows():
      stem_sub, stem_cont = [], []

      if(not pd.isnull(row['subject_list'])):
        # Removing punctuations from the sentence
        sentence = re.sub(r'[^0-9A-Za-z]+', ' ', row['subject_list'])
        for word in word_tokenize(sentence):
            # Replace the word with stem word 
            stem_word = PorterStemmer().stem(word)
            stem_cont.append(stem_word)
      else:
        stem_cont.append(' ')

      if(not pd.isnull(row['Title'])):
        # Removing punctuations from the sentence
        sentence = re.sub(r'[^0-9A-Za-z]+', ' ', row['Title'])
        for word in word_tokenize(sentence):
            # Replace the word with stem word 
            stem_word = PorterStemmer().stem(word)
            stem_sub.append(stem_word)
        else:
          stem_sub.append(' ')

      preprocessed_Subject.append(' '.join([word for word in stem_sub]))
      preprocessed_content.append(' '.join([word for word in stem_cont]))


  df['processed_subject'] = preprocessed_Subject
  df['processed_content'] = preprocessed_content
  return df

#Increase/Decrease weights based on response time
def check_corpus(df):
  urgency_list = ['soon', 'shortly','urgent', 'pressing','speedy', 
                  'prompt', 'straightaway', 'fast','promptly', 'quick', 'immediate', 'quickly', 'agile', 'ready']
  total_count = 0
  for word in urgency_list:
    temp_df = df['processed_subject'][df['processed_subject'].str.contains(word, na=False)]
    total_count += temp_df.count()
  return total_count

#Increase/Decrease weights based on response time
def check_response_time(df):
  df['Date'] =pd.to_datetime(df.Date_list)
  df_sorted = df.sort_values(by='Date')
  sorted_date_series = df_sorted['Date']
  time_diff_series = sorted_date_series.diff()
  in_digits = time_diff_series / np.timedelta64(1, 'D')
  min_difference = in_digits.min()
  if min_difference == 0:
    return 0
  else:
    return (1.0/min_difference)

#Compute Node Weight and exits to add more
def check_node_weight(weight_dict, df, reverse_node_num):
  for edge in weight_dict:
    idx = np.where((df['From']==reverse_node_num[edge[0]]) & (df['To']==reverse_node_num[edge[1]]))
    edge_mail_list = df.loc[idx]
    cleaned_edge_mail_list = clean_data(edge_mail_list)
    weight_dict[edge] += check_corpus(cleaned_edge_mail_list)
    weight_dict[edge] += check_response_time(cleaned_edge_mail_list)
    break
    

reverse_node_num = {}
for node in node_num_dict:
  reverse_node_num[node_num_dict[node]] = node

check_node_weight(weight_dict,features_df, reverse_node_num)
weight_list = []
for weight in weight_dict:
  weight_list.append(weight_dict[weight])
weight_dict = NormalizeDict(weight_dict,weight_list)
for weight in weight_dict:
  # print(weight_dict[weight])
  if np.isnan(weight_dict[weight]):
    weight_dict[weight] = 0
print("Node Weight Dictionary: ",weight_dict)

Node Weight Dictionary:  {(1, 14): 1.0, (14, 1): 4.818800196641807e-07, (2, 14): 1.2926736742109169e-08, (14, 2): 3.159868981404463e-07, (3, 12): 0.0, (12, 3): 7.181520412282871e-10, (3, 14): 3.447129797895778e-08, (14, 3): 2.4417169401761765e-07, (18, 3): 0.0, (7, 4): 1.4363040824565743e-09, (4, 14): 1.3644888783337456e-08, (14, 4): 2.0467333175006182e-07, (4, 19): 0.0, (19, 4): 0.0, (5, 14): 4.596173063861038e-08, (14, 5): 1.7451094601847377e-07, (5, 18): 7.181520412282871e-10, (18, 5): 2.8726081649131486e-09, (8, 6): 0.0, (6, 14): 5.027064288598011e-09, (14, 6): 1.4219410416320084e-07, (7, 14): 4.021651430878409e-08, (14, 7): 1.1993139088512394e-07, (8, 10): 1.4363040824565743e-09, (8, 14): 9.335976535967731e-09, (14, 8): 1.0269574189564505e-07, (14, 9): 1.005412857719602e-07, (14, 10): 9.264161331844904e-08, (11, 14): 1.3644888783337456e-08, (14, 11): 8.833270107107931e-08, (12, 14): 2.0108257154392043e-08, (14, 12): 8.474194086493789e-08, (13, 14): 1.1490432659652594e-08, (14, 13)

Find synonyms and antonyms

In [ ]:
from nltk.corpus import wordnet
synonyms = []
antonyms = []
def get_synonyms(word):
  for syn in wordnet.synsets(word):
    for l in syn.lemmas():
      synonyms.append(l.name())
      if l.antonyms():
          antonyms.append(l.antonyms()[0].name())
  return (synonyms,antonyms)


Execute Page rank and display ranks

In [ ]:
pr = PageRank(0.85, edges, 1e-6, 2, 22, weight_dict)
PageRank_vector = pr.pageRank()
page_rank_list = list(PageRank_vector)
print("Rank for the stop 20 user interactions are: ")
for i in range(1,20):
  print(reverse_node_num[i],':',page_rank_list[i])

Rank for the stop 20 user interactions are: 
suzanne.adams@enron.com : 1.4219773134477596e-08
nmann@erac.com : 5.003488605381242e-09
kathleen.carnahan@enron.com : 1.3728310636160166e-09
carlos.sole@enron.com : -4.624463750784985e-10
ben.jacoby@enron.com : -1.8987504575350728e-09
sheila.tweed@enron.com : -4.651666615576839e-09
ccampbell@kslaw.com : -5.888484019914445e-09
pthompson@akllp.com : -6.846020074885495e-09
reagan.rorschach@enron.com : -6.965712081756876e-09
roseann.engeldorf@enron.com : -6.925814746133082e-09
jkeffer@kslaw.com : -7.64396678736137e-09
gregg.penman@enron.com : -7.843453465480337e-09
heather.kroll@enron.com : -8.322221492965862e-09
kay.mann@enron.com : 1.0000001237778564
nwodka@bracepatt.com : -9.559038897303468e-09
kathleen.clark@enron.com : -9.878217582293818e-09
kent.shoemaker@ae.ge.com : -9.878217582293818e-09
fred.mitro@enron.com : -9.758525575422437e-09
jeffrey.hodge@enron.com : -1.0197396267284167e-08


Plot Graphs

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
from graphviz import Digraph

def generate_graph(rank_matrix, edge_list, node_num, weighted_vector, node_num_dict):
  dot = Digraph(comment='The Round Table')
  for edge in edge_list:
    dot.edge(str(reverse_node_num[edge[0]]),str(reverse_node_num[edge[1]]))
  dot.render('/content/drive/Shared drives/NLP Project/round-table15.gv', view=True)

reverse_node_num = {}
for node in node_num_dict:
  reverse_node_num[node_num_dict[node]] = node
generate_graph(PageRank_vector, edge_list, 21, weight_dict, reverse_node_num)
print("Check /content/drive/Shared drives/NLP Project/ folder for the graph")

Check /content/drive/Shared drives/NLP Project/ folder for the graph
